In [1]:
import re
import numpy as np
import pandas as pd
from IPython.display import display

#### Reading raw data files

In [2]:
accts_raw = pd.read_csv(r"../data/HI-Small_accounts.csv")
display(accts_raw.head(3))
display(accts_raw.dtypes)
txns_raw = pd.read_csv(r"../data/HI-Small_Trans.csv")
display(txns_raw.head(3))
display(txns_raw.dtypes)
display(txns_raw.describe())

,Bank Name,Bank ID,Account Number,Entity ID,Entity Name
0,Portugal Bank #4507,331579,80B779D80,80062E240,Sole Proprietorship #50438
1,Canada Bank #27,210,809D86900,800C998A0,Corporation #33520
2,UK Bank #33,21884,80812BE00,800C47F50,Partnership #35397


Bank Name         object
Bank ID            int64
Account Number    object
Entity ID         object
Entity Name       object
dtype: object

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0


Timestamp              object
From Bank               int64
Account                object
To Bank                 int64
Account.1              object
Amount Received       float64
Receiving Currency     object
Amount Paid           float64
Payment Currency       object
Payment Format         object
Is Laundering           int64
dtype: object

,From Bank,To Bank,Amount Received,Amount Paid,Is Laundering
count,5.078345e+06,5.078345e+06,5.078345e+06,5.078345e+06,5.078345e+06
mean,4.573057e+04,6.574456e+04,5.988726e+06,4.509273e+06,1.019427e-03
std,8.176562e+04,8.409299e+04,1.037183e+09,8.697728e+08,3.191219e-02
min,1.000000e+00,1.000000e+00,1.000000e-06,1.000000e-06,0.000000e+00
25%,1.190000e+02,4.259000e+03,1.833700e+02,1.844800e+02,0.000000e+00
50%,9.679000e+03,2.156800e+04,1.411010e+03,1.414540e+03,0.000000e+00
75%,2.862800e+04,1.223320e+05,1.234627e+04,1.229784e+04,0.000000e+00
max,3.563030e+05,3.562940e+05,1.046302e+12,1.046302e+12,1.000000e+00


In [3]:
display(txns_raw['Is Laundering'].value_counts())
print("Fraud Txn (%): ", len(txns_raw.loc[txns_raw['Is Laundering']==1])/ len(txns_raw) * 100)

Is Laundering
0    5073168
1       5177
Name: count, dtype: int64

Fraud Txn (%):  0.10194266045335636


In [4]:
'''
An ACH payment is an electronic fund transfer between bank accounts through the Automated Clearing House (ACH) network, commonly used for direct deposits, payroll, and bill payments. 
It is a reliable, low-cost method for moving money directly between financial institutions in the U.S. and internationally, with options for same-day settlement. 
'''
display(txns_raw['Payment Format'].value_counts())
display(txns_raw.loc[txns_raw['Is Laundering']==1]['Payment Format'].value_counts())

Payment Format
Cheque          1864331
Credit Card     1323324
ACH              600797
Cash             490891
Reinvestment     481056
Wire             171855
Bitcoin          146091
Name: count, dtype: int64

Payment Format
ACH            4483
Cheque          324
Credit Card     206
Cash            108
Bitcoin          56
Name: count, dtype: int64

#### Data Pre-processing/Cleaning

In [5]:
txns_df = txns_raw.copy(deep=True)
accts_df = accts_raw.copy(deep=True)

In [6]:
accts_df[['Entity Type', 'Entity Number']] = accts_df['Entity Name'].str.split(' #', expand=True)
accts_df.head(3)

,Bank Name,Bank ID,Account Number,Entity ID,Entity Name,Entity Type,Entity Number
0,Portugal Bank #4507,331579,80B779D80,80062E240,Sole Proprietorship #50438,Sole Proprietorship,50438
1,Canada Bank #27,210,809D86900,800C998A0,Corporation #33520,Corporation,33520
2,UK Bank #33,21884,80812BE00,800C47F50,Partnership #35397,Partnership,35397


In [7]:
accts_df['Entity Type'].value_counts()

Entity Type
Partnership            189683
Corporation            172351
Sole Proprietorship    149048
Country                  6692
Individual                740
Direct                     67
Name: count, dtype: int64

In [ ]:
## propose numerical hierarchical encoding of entity as node feature: Encodes of # of org size of account
## Country > Corporation ~ Direct > Partnership > Sole Proprietorship ~ Individual

In [8]:
txns_df = txns_df.rename(columns={
        'Account': 'From Account', 
        'Account.1': 'To Account'
    })

txns_df['Timestamp'] = pd.to_datetime(txns_df['Timestamp'], format='%Y/%m/%d %H:%M')

txns_df.head(3)

,Timestamp,From Bank,From Account,To Bank,To Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022-09-01 00:20:00,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022-09-01 00:20:00,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0
2,2022-09-01 00:00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0


In [9]:
# Checking for internal transactions (i.e. To Bank == From Bank)
txns_df['Internal'] = txns_df['From Bank'] == txns_df['To Bank']
display(txns_df['Internal'].value_counts())
display(txns_df.loc[txns_df['Is Laundering']==1]['Internal'].value_counts())

Internal
False    4387013
True      691332
Name: count, dtype: int64

Internal
False    5074
True      103
Name: count, dtype: int64

In [10]:
# Combine txns_df with accts_df
txns_accts_df = txns_df.merge(accts_df, how='left', left_on=('From Account', 'From Bank'), right_on=('Account Number', 'Bank ID'))\
                        .drop(columns=['Account Number', 'Bank ID'])\
                        .merge(accts_df, how='left', left_on=('To Account', 'To Bank'), right_on=('Account Number', 'Bank ID'), suffixes=(" (From)", " (To)"))\
                        .drop(columns=['Account Number', 'Bank ID'])

In [15]:
txns_accts_df['Entity Type Change'] = txns_accts_df['Entity Type (From)'] + " -> " + txns_accts_df['Entity Type (To)']
display(txns_accts_df['Entity Type Change'].value_counts())
display(txns_accts_df.loc[txns_accts_df['Is Laundering']==1]['Entity Type Change'].value_counts())

Entity Type Change
Partnership -> Partnership                    768915
Sole Proprietorship -> Sole Proprietorship    742915
Sole Proprietorship -> Partnership            565195
Corporation -> Corporation                    562993
Partnership -> Sole Proprietorship            529281
Sole Proprietorship -> Corporation            478948
Partnership -> Corporation                    467748
Corporation -> Partnership                    424977
Corporation -> Sole Proprietorship            420196
Country -> Country                             53436
Country -> Partnership                         33147
Partnership -> Country                         14566
Country -> Sole Proprietorship                  5054
Sole Proprietorship -> Country                  3042
Individual -> Corporation                       1289
Individual -> Partnership                       1159
Individual -> Sole Proprietorship               1086
Partnership -> Individual                        808
Individual -> Individual   

Entity Type Change
Partnership -> Partnership                    648
Corporation -> Partnership                    644
Partnership -> Corporation                    583
Corporation -> Sole Proprietorship            574
Partnership -> Sole Proprietorship            566
Corporation -> Corporation                    564
Sole Proprietorship -> Partnership            548
Sole Proprietorship -> Corporation            508
Sole Proprietorship -> Sole Proprietorship    496
Country -> Partnership                         16
Country -> Country                             12
Partnership -> Country                          8
Individual -> Partnership                       3
Country -> Sole Proprietorship                  3
Individual -> Sole Proprietorship               2
Corporation -> Country                          1
Individual -> Corporation                       1
Name: count, dtype: int64

In [44]:
data = []
with open(r'../data/HI-Small_Patterns.txt', 'r') as file:
    idx = 0
    for line in file:
        text = line.strip()
        begin_match = re.search(r"BEGIN LAUNDERING ATTEMPT", text)
        if begin_match:
            pattern_type = re.split(r" - ", text)[1]
            pattern_type = pattern_type + f" ({str(idx)})"
            idx += 1
        else:
            if text == "" or "END LAUNDERING ATTEMPT" in text:
                continue
            row_values = re.split(r",", text)
            row_values.append(pattern_type)
            data.append(row_values)

patterns_df = pd.DataFrame(data, columns=
    ['Timestamp',
    'From Bank',
    'From Account',
    'To Bank',
    'To Account',
    'Amount Received',
    'Receiving Currency',
    'Amount Paid',
    'Payment Currency',
    'Payment Format',
    'Is Laundering',
    'Pattern ID']
)

patterns_df['Timestamp'] = pd.to_datetime(patterns_df['Timestamp'], format='%Y/%m/%d %H:%M')
patterns_df['Pattern Type'] = patterns_df['Pattern ID'].str.split(':').str[0]
patterns_df.head(3)

,Timestamp,From Bank,From Account,To Bank,To Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Pattern ID,Pattern Type
0,2022-09-01 00:06:00,021174,800737690,012,80011F990,2848.96,Euro,2848.96,Euro,ACH,1,FAN-OUT: Max 16-degree Fan-Out (0),FAN-OUT
1,2022-09-01 04:33:00,021174,800737690,020,80020C5B0,8630.40,Euro,8630.40,Euro,ACH,1,FAN-OUT: Max 16-degree Fan-Out (0),FAN-OUT
2,2022-09-01 09:14:00,021174,800737690,020,80006A5E0,35642.49,Yuan,35642.49,Yuan,ACH,1,FAN-OUT: Max 16-degree Fan-Out (0),FAN-OUT
